# KNN (K-Nearest-Neighbors)


KNN is a simple concept: define some distance metric between the items in your dataset, and find the K closest items. You can then use those items to predict some property of a test item, by having them somehow "vote" on it.

As an example, let's look at the MovieLens data. We'll try to guess the rating of a movie by looking at the 10 movies that are closest to it in terms of genres and popularity.

To start, we'll load up every rating in the data set into a Pandas DataFrame:


In [102]:
# Importing the pandas library for data manipulation and analysis.
import pandas as pd

# Defining the column names for the ratings data.
r_cols = ["user_id", "movie_id", "rating"]

# Reading the ratings data from a tab-separated file.
# 'sep' specifies tab character as delimiter.
# 'names' assigns the column names.
# 'usecols' selects the first three columns (user_id, movie_id, rating).
ratings = pd.read_csv("../ml-100k/u.data", sep="\t", names=r_cols, usecols=range(3))

# Displaying the first 50 rows of the DataFrame.
# This helps in quickly checking the top rows of the dataset to ensure it looks as expected.
ratings.head()

,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


Now, we'll group everything by movie ID, and compute the total number of ratings (each movie's popularity) and the average rating for every movie:


In [103]:
# Importing numpy for numerical operations
import numpy as np

# Assuming 'ratings' is a DataFrame that has already been loaded,
# containing columns 'user_id', 'movie_id', and 'rating'.

# Grouping the 'ratings' DataFrame by 'movie_id'.
# This organizes the data such that each group corresponds to a unique movie.
movieProperties = ratings.groupby("movie_id").agg({"rating": [np.size, np.mean]})

# The 'agg' function is used for aggregation. Here, it aggregates the 'rating' column of each group (movie) in two ways:
# 'np.size' counts the number of ratings for each movie, giving an idea of its popularity or how widely it's been watched.
# 'np.mean' calculates the average rating for each movie, indicating its general reception or quality as perceived by users.

# Displaying the first five rows of the aggregated DataFrame.
# This gives a quick look at the movie properties (number of ratings and average rating) for the first few movies.
movieProperties.head()

/var/folders/dn/2j4k_qd53gdgn8wfhr4djdrr0000gq/T/ipykernel_5748/4050712149.py:9: FutureWarning: The provided callable <function mean at 0x104ab2430> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movieProperties = ratings.groupby("movie_id").agg({"rating": [np.size, np.mean]})


rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

The raw number of ratings isn't very useful for computing distances between movies, so we'll create a new DataFrame that contains the normalized number of ratings. So, a value of 0 means nobody rated it, and a value of 1 will mean it's the most popular movie there is.


In [104]:
# Assuming 'movieProperties' is a DataFrame created earlier, which contains aggregated data for each movie.

# Extracting just the 'size' column from the 'movieProperties' DataFrame, which represents the number of ratings per movie.
# This is wrapped in a new DataFrame to maintain a tabular structure.
movieNumRatings = pd.DataFrame(movieProperties["rating"]["size"])

# Normalizing the number of ratings for each movie.
# The lambda function in 'apply' scales each value to a range between 0 and 1.
# This is done by subtracting the minimum number of ratings from each value and then dividing by the range of ratings.
# Normalization formula: (x - min(x)) / (max(x) - min(x))
movieNormalizedNumRatings = movieNumRatings.apply(
    lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))
)
# x:
# In the context of a pandas DataFrame or Series, x typically represents each individual element or value in that DataFrame or Series.
# When you use a function like apply in pandas, x will be each value that the function iterates over.
# For example, if you have a Series of movie ratings, x would be each rating in that Series as the function processes them one by one.
# np.min(x):
# np.min(x) is a function call to NumPy's min function, which calculates the minimum value of x.
# In a DataFrame or Series, np.min(x) would return the smallest value found in that DataFrame or Series.
# This value is used in normalization to shift the scale of the data so that the smallest value becomes 0.
# np.max(x):
# Similarly, np.max(x) is a call to NumPy's max function, which finds the maximum value of x.
# In a DataFrame or Series, it returns the largest value present.
# In normalization, this is used to understand the range of the data. It helps in scaling the largest value in the data to 1.

# Displaying the first five rows of the normalized number of ratings.
# This shows the normalized ratings for the first few movies.
movieNormalizedNumRatings.head()

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


Now, let's get the genre information from the u.item file. The way this works is there are 19 fields, each corresponding to a specific genre - a value of '0' means it is not in that genre, and '1' means it is in that genre. A movie may have more than one genre associated with it.

While we're at it, we'll put together everything into one big Python dictionary called movieDict. Each entry will contain the movie name, list of genre values, the normalized popularity score, and the average rating for each movie:


In [105]:
# 'movieDict' will be our dictionary to store all movie-related data
movieDict = {}

# Opening the 'u.item' file in read mode. The file encoding is specified as "ISO-8859-1".
with open(r"../ml-100k/u.item", encoding="ISO-8859-1") as f:
    # Iterating through each line in the file
    for line in f:
        # Splitting each line at '|' and stripping the newline character at the end.
        fields = line.rstrip("\n").split("|")

        # Extracting the movie ID and converting it to an integer.
        movieID = int(fields[0])

        # Extracting the movie name.
        name = fields[1]

        # Extracting the genre information. Genres are in fields 5 to 24.
        # Each genre field is a '0' or '1', indicating absence/presence of the movie in that genre.
        genres = fields[5:25]
        # print(genres)

        # Converting very element in genres list from string to integer using map.
        genres = map(int, genres)
        # print(list(genres))

        # Constructing a tuple with the movie name, an array of genre values,
        # the normalized number of ratings, and the average rating.
        # These values are accessed from 'movieNormalizedNumRatings' and 'movieProperties' DataFrames.
        movieDict[movieID] = (
            name,
            np.array(list(genres)),
            movieNormalizedNumRatings.loc[movieID].get("size"),
            movieProperties.loc[movieID].rating.get("mean"),
        )

For example, here's the record we end up with for movie ID 1, "Toy Story":


In [106]:
# Iterating through the first 10 movie IDs in the 'movieDict'.
for i in range(1, 11):
    # Printing the movie ID and its corresponding data (name, genres, normalized number of ratings, average rating).
    print(i, movieDict[i])

1 ('Toy Story (1995)', array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.7735849056603774, 3.8783185840707963)
2 ('GoldenEye (1995)', array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), 0.22298456260720412, 3.2061068702290076)
3 ('Four Rooms (1995)', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), 0.15265866209262435, 3.033333333333333)
4 ('Get Shorty (1995)', array([0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.3567753001715266, 3.550239234449761)
5 ('Copycat (1995)', array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), 0.1457975986277873, 3.302325581395349)
6 ('Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.04288164665523156, 3.576923076923077)
7 ('Twelve Monkeys (1995)', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]), 0.6706689536878216, 3.798469387755102)
8 ('Babe (1995)', array([0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 

Now let's define a function that computes the "distance" between two movies based on how similar their genres are, and how similar their popularity is. Just to make sure it works, we'll compute the distance between movie ID's 2 and 4:


In [107]:
# Importing the spatial module from scipy for distance calculations
from scipy import spatial

# Defining the function ComputeDistance to calculate the distance between two movies
def ComputeDistance(a, b):
    # Extracting the genre array for both movies
    genresA = a[1]
    genresB = b[1]
    
    # Calculating the cosine distance between the genre arrays of the two movies
    # Cosine distance is a measure of similarity between two non-zero vectors 
    # of an inner product space that measures the cosine of the angle between them.
    genreDistance = spatial.distance.cosine(genresA, genresB)
    
    # Extracting the normalized popularity scores for both movies
    popularityA = a[2]
    popularityB = b[2]
    
    # Calculating the absolute difference in popularity scores between the two movies
    # This represents how different the movies are in terms of their popularity.
    popularityDistance = abs(popularityA - popularityB)
    
    # The total 'distance' between the two movies is a combination of their genre and popularity distances
    return genreDistance + popularityDistance

# Computing the distance between movies with ID's 2 and 4 using the defined function
ComputeDistance(movieDict[2], movieDict[4])

0.8004574042309892

Remember the higher the distance, the less similar the movies are. Let's check what movies 2 and 4 actually are - and confirm they're not really all that similar:


In [108]:
print(movieDict[2])
print(movieDict[4])

('GoldenEye (1995)', array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), 0.22298456260720412, 3.2061068702290076)
('Get Shorty (1995)', array([0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.3567753001715266, 3.550239234449761)


Now, we just need a little code to compute the distance between some given test movie (Toy Story, in this example) and all of the movies in our data set. When the sort those by distance, and print out the K nearest neighbors:


In [109]:
# Importing the operator module, which provides functions for performing comparisons
import operator

# Defining a function to find the K nearest neighbors of a specific movie based on their 'distance'
def getNeighbors(movieID, K):
    distances = []  # Initializing an empty list to store the distances of each movie from the given movie

    # Iterating through each movie in the movieDict dictionary
    for movie in movieDict:
        # Ensuring the movie is not compared with itself
        if movie != movieID:
            # Calculating the distance between the given movie and the current movie in the loop
            dist = ComputeDistance(movieDict[movieID], movieDict[movie])
            # Adding the movie ID and its calculated distance to the distances list
            distances.append((movie, dist))

    # Sorting the distances list in ascending order based on the distance value
    # 'operator.itemgetter(1)' is used to sort by the second item in each tuple (the distance)
    distances.sort(key=operator.itemgetter(1))

    # Creating a list to store the IDs of the K nearest neighbors
    neighbors = []
    # Looping through the sorted list to pick the first K movies
    for x in range(K):
        # Appending the ID of each nearest neighbor to the neighbors list
        neighbors.append(distances[x][0])

    # Returning the list of the K nearest neighbors' IDs
    return neighbors

# Setting the number of neighbors (K) to find to 10
K = 10
# Initializing a variable to calculate the average rating
avgRating = 0
# Getting the 10 nearest neighbors of the movie with ID 1
neighbors = getNeighbors(1, K)

# Iterating over each neighbor (movie ID) in the neighbors list
for neighbor in neighbors:
    # Adding the average rating of each neighbor movie to avgRating
    avgRating += movieDict[neighbor][3]
    # Printing the movie name and its average rating for each neighbor
    print(movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))

# Calculating the average rating of the K nearest neighbors
avgRating /= K

Liar Liar (1997) 3.156701030927835
Aladdin (1992) 3.8127853881278537
Willy Wonka and the Chocolate Factory (1971) 3.6319018404907975
Monty Python and the Holy Grail (1974) 4.0664556962025316
Full Monty, The (1997) 3.926984126984127
George of the Jungle (1997) 2.685185185185185
Beavis and Butt-head Do America (1996) 2.7884615384615383
Birdcage, The (1996) 3.4436860068259385
Home Alone (1990) 3.0875912408759123
Aladdin and the King of Thieves (1996) 2.8461538461538463


While we were at it, we computed the average rating of the 10 nearest neighbors to Toy Story:


In [110]:
avgRating

3.3445905900235564

How does this compare to Toy Story's actual average rating?


In [111]:
movieDict[1][3]

3.8783185840707963

In [112]:
# 'avgRating' is the average rating of the K nearest neighbors of the movie with ID 1.
# 'movieDict' is a dictionary where each key is a movie ID and the value is a tuple containing movie details.
# 'movieDict[1][3]' accesses the average rating of the movie with ID 1 from the dictionary.
# The fourth element (index 3) in each tuple is the average rating of the movie.

# Calculating the relative difference between the average rating of the K nearest neighbors 
# and the average rating of the movie itself.
relativeDifference = (avgRating - movieDict[1][3]) / movieDict[1][3]

# The result, 'relativeDifference', is a measure of how the average rating of the nearest neighbors 
# compares to the movie's own average rating.
# A positive value indicates the neighbors are rated higher on average, 
# a negative value indicates they are rated lower, 
# and a value close to zero indicates similar average ratings.

Not too bad!


## Activity


Our choice of 10 for K was arbitrary - what effect do different K values have on the results?

Our distance metric was also somewhat arbitrary - we just took the cosine distance between the genres and added it to the difference between the normalized popularity scores. Can you improve on that?
